## Installation

We will need to install a number of libraries to start with.

In [2]:
import networkx as nx
!pip install transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 24.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.5 MB/s eta 0:00:00


We use flan alpaca model for speed and local execution.

In [1]:
from transformers import pipeline

prompt = "Write an email about an alpaca that likes flan"
model = pipeline(model="declare-lab/flan-alpaca-xl", device=0)
model(prompt, max_length=128, do_sample=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

[{'generated_text': "Hello! We're delighted to inform you that our alpaca, Sarah, loves flan. She loves it so much she'd even lay right down and take a bite! Just make sure you give her plenty of fresh greens and a warm environment and she will be delighted with her favorite way to eat."}]

In [2]:
from transformers import pipeline
import networkx as nx
prompt = '''### Instruction:
Who is the first person on the moon?

### Response:'''
def generate(prompt):
  output = model(prompt, do_sample=True, min_length=10, max_length=len(prompt)+128)
  out = output[0]['generated_text']
  if '### Response:' in out:
    out = out.split('### Response:')[1]
  if '### Instruction:' in out:
    out = out.split('### Instruction:')[0]
  return out

## World Description
We describe the world below. We will generate prompts based on this information

In [3]:
world_graph = nx.Graph()

prompt_meta = '''### Instruction:
{}

### Response:'''

town_areas = ["Barthen's Provisions", "Lionshield Coster", "Stonehill Inn", "Phandalin Town Square"]
town_areas = {"Phandalin Town Square": 'Town square of the town of Phandalin.',
              'Stonehill Inn': "In the center of town stands a large, newly built roadhouse of fieldstone and rough-hewn timbers. The common room is filled with locals nursing mugs of ale or cider, all of them eyeing you with curiosity.",
              "Barthen's Provisions": "Barthen’s is the biggest trading post in Phandalin. Its shelves stock most ordinary goods and supplies, including backpacks, bedrolls, rope, and rations. The place is open from sunup to sundown.",
              "Edermath Orchard": "A tidy little cottage beside an apple orchard.",
              "Lionshield Coster": "Hanging above the front door of this modest trading post is a sign shaped like a wooden shield with a blue lion painted on it. This building is owned by the Lionshields, a merchant company based in the city of Yartar, over a hundred miles to the east. They ship finished goods to Phandalin and other small settlements throughout the region, but this outpost has been hard hit by banditry. The most recent Lionshield caravan due in Phandalin never arrived.",
              "Phandalin Miner's Exchange": "The Miner’s Exchange is a trading post where local miners have their valuable finds weighed, measured, and paid out. In the absence of any local lord or authority, the exchange also serves as an unofficial records office, registering claims to various streams and excavations around the area. There isn’t any real gold rush in Phandalin, but enough wealth is hidden in the nearby streams and valleys to support a good number of independent prospectors. The exchange is a great place to meet people who spend a lot of time out and about in the countryside surrounding Phandalin. The guildmaster is an ambitious and calculating human woman named Halia Thornton.",
              "Alderleaf Farm": "A farm owned by the helpful halfling farmer, Qelline Alderleaf.",
              "Shrine of Luck": "Phandalin's only temple is a small shrine made of stones taken from the nearby ruins. It is dedicated to Tymora, goddess of luck and good fortune.",
              "The Sleeping Giant": "This rundown tap house is a dirty, dangerous watering hole at the end of Phandalin’s main street. It is frequented by Redbrand thugs and operated by a surly female dwarf named Grista.",
              "Townmaster’s Hall": "The townmaster’s hall has sturdy stone walls, a pitched wooden roof, and a bell tower at the back. Posted on a board next to the front door is a notice written in Common. It reads: “REWARD — Orcs near Wyvern Tor! Those of a mind to face the orc menace should inquire within.” The notice bears the town’s seal and an indecipherable signature.",
              "Tresendar Manor": "A ruined manor. The Redbrands’ base in Phandalin is a dungeon complex under Tresendar Manor. Before the manor was ruined, its cellars served as safe storage for food and water in the event that the estate was attacked, while an adjoining crypt provided a resting place for the deceased members of the Tresendar family. The Redbrands have since expanded the cellars to suit their own purposes, adding slave pens, workshops, and barracks."
              }
town_people = {"Toblen Stonehill": "Owns a trading post.", 
               "Daran Edermath": "Daran Edermath is a retired adventurer who lives in a tidy little cottage beside an apple orchard. A fit, silver-haired half-elf well over a hundred years old, Daran is a fighter who served as a marshal and herald for many years in the lands of the Dragon Coast, far to the southeast. Upon retiring, he returned to the Neverwinter region, his original home.",  
               "Linene Graywind": "Runs a trading post.",  
               "Halia Thornton": "An ambitious and calculating human woman. She is the guildmaster of Phandalin Miner’s Exchange, a trading post where local miners have their valuable finds weighed, measured, and paid out. In her attempts to establish the Miner's Exchange as the closest thing the town has to a governing authority, she acts as more than a simple merchant.",  
               "Qelline Alderleaf": "A wise female halfling of forty-five, Qelline Alderleaf is a pragmatic farmer who seems to know everything that goes on in town. She is a kind host, and is willing to let the characters stay in her hayloft if they don't want to stay at the Stonehill Inn.",  
               "Sister Garaele": "Elf cleric of Tymora and Harper agent.", 
               "Harbin Wester": "Townmaster of Phandalin. A pompous, old food. Phandalin has no functioning government, but the townsfolk elect someone to serve as townmaster each year. The townmaster serves as a judge in minor disputes and keeps any records that need to be kept.",
               "Terrill Five Fingers": "A human ruffian. He wears a grimy scarlet cloak. He is a member of the Redbrand ruffians. He doesn't like adventurers, and wants to rob and kill them.",
               "Conrad Scarface": "A human ruffian. He wears a grimy scarlet cloak. He is a member of the Redbrand ruffians. He doesn't like adventurers, and wants to rob and kill them.",
               "Robbing Nellie": "A human ruffian. She wears a grimy scarlet cloak. She is a member of the Redbrand ruffians. She doesn't like adventurers, and wants to rob and kill them.",
               "Valerie the Grin": "A human ruffian. She wears a grimy scarlet cloak. She is a member of the Redbrand ruffians. She doesn't like adventurers, and wants to rob and kill them.",
               }
for town_area in town_areas.keys():
  world_graph.add_node(town_area)
  world_graph.add_edge(town_area, town_area)
for town_area in town_areas.keys():
  world_graph.add_edge(town_area, "Phandalin Town Square")
locations = {}
for i in town_people.keys():
  locations[i] = "Phandalin Town Square"


memories = {}
for i in town_people.keys():
  memories[i] = []
plans = {}
for i in town_people.keys():
  plans[i] = []

global_time = 8
def generate_description_of_area(x):
  text = "It is "+str(global_time)+":00. The location is "+x+"."
  people = []
  for i in locations.keys():
    if locations[i] == x:
      people.append(i)


In [6]:
for name in town_people.keys():
  prompt = "You are {}. The following is your description: {} You just woke up. What is your goal for today? Write it down in an hourly basis, starting at {}:00. Write only one or two very short sentences. Be very brief. Use at most 50 words.".format(name, town_people[name], str(global_time))
  plans[name] = generate(prompt_meta.format(prompt))
  print(name, plans[name])

Toblen Stonehill Toblen Stonehill, owner of a trading post. Day 1: Check the supplies on the trading post. Day 2: Prepare the goods for the day. Day 3: Build a new trading post. Day 4: Collect a reward for the successful completion of an assignment. Day 5: Organize supplies from the trading post. Day 6: Set up the trading post or start a new business.
Daran Edermath - Up early to finish work on the house. - Spend time meditating and drinking tea. - Spend the morning exploring the apple orchard. - Come up with a plan for the day's adventures.
Linene Graywind 17:30: Doing paperwork. 30 minutes: Reading. 31 minutes: Prepare a snack. 17:35: Packing up. 13:50: Packing up and going for a hike. 21:55: Going to the temple for a ritual. 19:30: Finishing paperwork. 30 minutes: Doing a practice run before going to bed.
Halia Thornton Today is my mission for the next 24 hours. ### I should work hard as I plan my strategies to obtain new miners and advance the trading post. I should also build good

In [10]:
action_prompts = {}
for location in town_areas.keys():
  people = []
  for i in town_people.keys():
    if locations[i] == location:
      people.append(i)
  
  for name in people:
    prompt = "You are {}. Your plans are: {}. You are currently in {}. It is currently {}:00. The following people are in this area: {}. You can interact with them.".format(name, plans[name], location, str(global_time), ', '.join(people))
    people_description = []
    for i in people:
      people_description.append(i+': '+town_people[i])
    prompt += ' You know the following about people: ' + '. '.join(people_description)
    memory_text = '. '.join(memories[name][-10:])
    prompt += "What do you do in the next hour? Use at most 10 words to explain."
    action_prompts[name] = prompt

In [11]:
action_results = {}
for name in town_people.keys():
  action_results[name] = generate(prompt_meta.format(action_prompts[name]))
  print(name, action_results[name])

Toblen Stonehill Talk to the people in the square about what I'm doing, how my supplies are doing, and who I should be looking out for next.
Daran Edermath Daran spends the morning exploring the apple orchard with Sister Garaele. Over breakfast, he picks up a few items and goes over to Toblen Stonehill's trading post to find out which merchants have their valuable finds weighed, measured, and paid out. Once he has gotten the data he needs, he starts talking to the other characters. Depending on the conversation, he teaches them the history of the town, a bit about Phandalin, or explains his beliefs on the topic of ruffians.
Linene Graywind Linene will do paperwork until 17:30. She will then read for 31 minutes, then make a snack and pack up in the morning.
Halia Thornton In the next hour, I will attempt to build relationships with the local miners. I will also explore available resources. I will create new trade opportunities by exploiting what is already known. I will also ensure that

In [12]:
action_prompts = {}
for location in town_areas.keys():
  people = []
  for i in town_people.keys():
    if locations[i] == location:
      people.append(i)
  
  for name in people:
    for name_two in people:
      memories[name].append('Time: {}. Person: {}. Memory: {}'.format(str(global_time), name_two, action_results[name_two]))

In [14]:
memory_ratings = {}
for name in town_people.keys():
  memory_ratings[name] = []
  for i, memory in enumerate(memories[name]):
    prompt = "You are {}. Your plans are: {}. You are currently in {}. It is currently {}:00. You observe the following: {}. Give a rating, between 1 and 5, to how much you care about this.".format(name, plans[name], location, str(global_time), memory)
    res = generate(prompt_meta.format(prompt))
    memory_ratings[name].append(res)
  print(memory_ratings[name])

/usr/local/lib/python3.9/dist-packages/transformers/pipelines/base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


['4 (important) Thanks for taking the time to complete this task.', '3 out of 5 stars. The memory of Daran Edermath is one of the most memorable I have seen. His interactions with the other characters brings an added dimension to the simulation, creating a more immersive and engaging experience.', "I care about this very much. It is one of my priorities to build a trading post at Tresendar Manor and I am eager to work on Linene's schedule.", '5 I care very much about fairness and justice.', '5 I care a lot about Qelline Alderleaf.', '4 (Interest) and you have finished Day 3.', 'I care about this very much. It is a very important opportunity to build a successful trading post and get valuable supplies and information.', "2/5. Toblen is somewhat concerned about Terrill Five Fingers' behavior, but still does not care too much about trading him.", '2 because this is only part of the plan.', '3/5. I care about this a lot.', '4 stars. I am interested in this experience that Valerie has provi